In [ ]:
# default_exp conversion

# data

> This module defines typecasting and conversion functions for easily manipulating and generating data. The most powerful methods are `cast` and `convert`. Other utilities include `RNG` and `fill`, as well as extensions of binary logic to continuous variables such as `XOR`, and encodings such as gray code.

In [ ]:
#hide
from nbdev import *
from nbdev.imports import *
from nbdev.export import *
from nbdev.sync import *
from nbdev.showdoc import *

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import typing
import numpy as np
from typing import Optional, Tuple, Dict, Callable, Union, Mapping, Sequence, Iterable
from functools import partial
import warnings

C:\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


## Type casting

We begin with some motivating definitions and examples before building up to the the final typecasting function `cast`. Thee data types considered here are given as a typehint `data` and dictionary `types`. 

In [ ]:
#export
data = Union[int,float,list,tuple,str,dict,set,np.ndarray]
types={t:t for t in [int,float,list,tuple,str,dict,set,np.ndarray]}

We start with a simple try except conversion which guarantees the object is at least *treated* as the desired type, even if it can't be directly converted (yet).

In [ ]:
#export
def __cast(to : data, obj : data) -> data: 
    '''
    Typecasts `obj` to datatype `to`.
    First inner typecasting function.
    Fallback case of more complex casting behavior.
    '''
    try:
        return to(obj)
    except:
        return typing.cast(to,obj)

In [ ]:
show_doc(__cast)

<h4 id="__cast" class="doc_header"><code>__cast</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>__cast</code>(**`to`**:`Union`\[`int`, `float`, `list`, `tuple`, `str`, `dict`, `set`, `ndarray`\], **`obj`**:`Union`\[`int`, `float`, `list`, `tuple`, `str`, `dict`, `set`, `ndarray`\])

Typecasts `obj` to datatype `to`.
First inner typecasting function.
Fallback case of more complex casting behavior.

Next, we will create a global dict `rules` keyed by type that is valued by a partially evaluated `__cast` on that type. This allows us to modify the future behavior of the cast if desired. To understand partial evaluation, consider:

In [ ]:
p=partial(lambda x,y: (x,y),1)
p(0)

(1, 0)

In [ ]:
#export
rules={t:partial(__cast,t) for t in types}
rules[np.ndarray]=np.array #use the numpy function 
rules[None]=lambda t:t #and return the object in case nothing given

In [ ]:
rules[None]('no arg type, returns whatever we give it')

'no arg type, returns whatever we give it'

In [ ]:
print(rules[int](0.5))
print(rules[int]([0.5]))
rules[np.ndarray](0)

0
[0.5]


array(0)

The list input, array, and None would have failed without this treatment. We now define another intermediary `_cast` to do this so we can treat it as a callable and switch the order of the arguments to the more pythonic `obj`, `to`.

In [ ]:
#export
def cast_(obj : data,to : data) -> data:
    '''
    Second inner typecast function using global rulebook `rules`.
    '''
    global rules
    return rules[to](obj)

In [ ]:
show_doc(cast_)

<h4 id="cast_" class="doc_header"><code>cast_</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>cast_</code>(**`obj`**:`Union`\[`int`, `float`, `list`, `tuple`, `str`, `dict`, `set`, `ndarray`\], **`to`**:`Union`\[`int`, `float`, `list`, `tuple`, `str`, `dict`, `set`, `ndarray`\])

Second inner typecast function using global rulebook [`rules`](/sidis/conversion.html#rules).

With this formulation we can update the rules and get new casting behavior.

In [ ]:
rules[np.ndarray]=lambda t:print('changed')

In [ ]:
cast_(0,np.ndarray)

changed


And change it back.

In [ ]:
rules[np.ndarray]=np.array
cast_(0,np.ndarray)

array(0)

## Wrapping

The type casting above is simple. Often times it is convenient to "wrap" the desired type over the input type. We will now construct a `wrap` function which does this, depending on if the object is an iterable or not.

In [ ]:
#export
isiter = lambda t: hasattr(t,'__iter__')
iterables=[t for t in types if isiter(t)]
numbers=[t for t in types if t not in iterables]

In [ ]:
#export
def typestr(x : data):
    '''
    Parses the string of the input type for readability.
    '''
    if type(x) is type: #if passing type itself
        s=x
    else: #otherwise get type of obj
        s=type(x)
    return str(s).split('<')[-1].split('>')[0].split('class')[-1].split('\'')[1]

In [ ]:
show_doc(typestr)
print([typestr(t) for t in iterables])
print([typestr(t) for t in numbers])

<h4 id="typestr" class="doc_header"><code>typestr</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>typestr</code>(**`x`**:`Union`\[`int`, `float`, `list`, `tuple`, `str`, `dict`, `set`, `ndarray`\])

Parses the string of the input type for readability.

['list', 'tuple', 'str', 'dict', 'set', 'numpy.ndarray']
['int', 'float']


The following conversions between the two types demonstrate these meanings:

In [ ]:
#iters<->iters
for i in iterables:
    for j in [[0,1],(0,1),'01',{0:1},{0,1},np.array([0,1])]:
        print(j,' to ',typestr(i),' : ',cast_(j,i))

[0, 1]  to  list  :  [0, 1]
(0, 1)  to  list  :  [0, 1]
01  to  list  :  ['0', '1']
{0: 1}  to  list  :  [0]
{0, 1}  to  list  :  [0, 1]
[0 1]  to  list  :  [0, 1]
[0, 1]  to  tuple  :  (0, 1)
(0, 1)  to  tuple  :  (0, 1)
01  to  tuple  :  ('0', '1')
{0: 1}  to  tuple  :  (0,)
{0, 1}  to  tuple  :  (0, 1)
[0 1]  to  tuple  :  (0, 1)
[0, 1]  to  str  :  [0, 1]
(0, 1)  to  str  :  (0, 1)
01  to  str  :  01
{0: 1}  to  str  :  {0: 1}
{0, 1}  to  str  :  {0, 1}
[0 1]  to  str  :  [0 1]
[0, 1]  to  dict  :  [0, 1]
(0, 1)  to  dict  :  (0, 1)
01  to  dict  :  01
{0: 1}  to  dict  :  {0: 1}
{0, 1}  to  dict  :  {0, 1}
[0 1]  to  dict  :  [0 1]
[0, 1]  to  set  :  {0, 1}
(0, 1)  to  set  :  {0, 1}
01  to  set  :  {'1', '0'}
{0: 1}  to  set  :  {0}
{0, 1}  to  set  :  {0, 1}
[0 1]  to  set  :  {0, 1}
[0, 1]  to  numpy.ndarray  :  [0 1]
(0, 1)  to  numpy.ndarray  :  [0 1]
01  to  numpy.ndarray  :  01
{0: 1}  to  numpy.ndarray  :  {0: 1}
{0, 1}  to  numpy.ndarray  :  {0, 1}
[0 1]  to  numpy.ndarr

In [ ]:
#numbers<->numbers
for i in numbers:
    for j in [0,0.1]:
        print(j,' to ',typestr(i),' : ',cast_(j,i))

0  to  int  :  0
0.1  to  int  :  0
0  to  float  :  0.0
0.1  to  float  :  0.1


In [ ]:
#numbers->iterables
for i in numbers:
    for j in [[0,0.1],(0,0.1),'01',{0:0.1},{0,0.1},np.array([0,0.1])]:
        print(j,' to ',typestr(i),' : ',cast_(j,i))

[0, 0.1]  to  int  :  [0, 0.1]
(0, 0.1)  to  int  :  (0, 0.1)
01  to  int  :  1
{0: 0.1}  to  int  :  {0: 0.1}
{0, 0.1}  to  int  :  {0, 0.1}
[0.  0.1]  to  int  :  [0.  0.1]
[0, 0.1]  to  float  :  [0, 0.1]
(0, 0.1)  to  float  :  (0, 0.1)
01  to  float  :  1.0
{0: 0.1}  to  float  :  {0: 0.1}
{0, 0.1}  to  float  :  {0, 0.1}
[0.  0.1]  to  float  :  [0.  0.1]


In [ ]:
#iterables->numbers
for i in iterables:
    for j in [0,0.1]:
        print(j,' to ',typestr(i),' : ',cast_(j,i))

0  to  list  :  0
0.1  to  list  :  0.1
0  to  tuple  :  0
0.1  to  tuple  :  0.1
0  to  str  :  0
0.1  to  str  :  0.1
0  to  dict  :  0
0.1  to  dict  :  0.1
0  to  set  :  0
0.1  to  set  :  0.1
0  to  numpy.ndarray  :  0
0.1  to  numpy.ndarray  :  0.1


As can be seen, the cross-conversions are nonexistent. We change this with the following rules:

> iters->numbers : convert every element in the iter

> numbers->iters : wrap the iterable type around the number

For the special case of dictionaries, which are multi-valued, we convert the *values* to the number type, and not the keys (the keys are obtainable from the iterable<->iterable mappings above). Similarly, for numbers->dict, we key by the number itself. This is a convention but it is nonambiguous when iterating over the number, as it will return the number. In summary:

> dict -> number : convert every value (not key) to number

> number -> dict : key by number

In [ ]:
#export
def cast(obj : data, to : data = None) -> data:
    '''
    Universal typecasting function.
    Typecasts the `obj` to the desired datatype `to`,
    with support for wrapping of various types.
    '''
    t=type(obj)
    to_iter=isiter(to)
    in_iter=isiter(t)
    if to_iter==in_iter:
        return cast_(obj,to)
    elif isiter(t): #if it's an iterable, convert all the elements, then convert back
        if t is dict: 
            return {k:cast_(v,to) for k,v in obj.items()}
        else:# t in iterables:
            return cast_([cast_(i,to) for i in obj],t)
    else: #otherwise, wrap around the exterior. do this manually for each type, unfortunately
        if to is list:
            return [obj]
        elif to is tuple:
            return (obj,)
        elif to is str:
            return f'{obj}'
        elif to is dict:
            return {obj:obj}
        elif to is set:
            return {obj}
        elif to is np.ndarray:
            return np.array(obj).astype(to)

In [ ]:
show_doc(cast)
for n in [1,1.5,2.3]:
    for t in types:
        print(f'From {n} to {typestr(t)}: {cast(n,t)}')

<h4 id="cast" class="doc_header"><code>cast</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>cast</code>(**`obj`**:`Union`\[`int`, `float`, `list`, `tuple`, `str`, `dict`, `set`, `ndarray`\], **`to`**:`Union`\[`int`, `float`, `list`, `tuple`, `str`, `dict`, `set`, `ndarray`\]=*`None`*)

Universal typecasting function.
Typecasts the `obj` to the desired datatype `to`,
with support for wrapping of various types.

From 1 to int: 1
From 1 to float: 1.0
From 1 to list: [1]
From 1 to tuple: (1,)
From 1 to str: 1
From 1 to dict: {1: 1}
From 1 to set: {1}
From 1 to numpy.ndarray: 1
From 1.5 to int: 1
From 1.5 to float: 1.5
From 1.5 to list: [1.5]
From 1.5 to tuple: (1.5,)
From 1.5 to str: 1.5
From 1.5 to dict: {1.5: 1.5}
From 1.5 to set: {1.5}
From 1.5 to numpy.ndarray: 1.5
From 2.3 to int: 2
From 2.3 to float: 2.3
From 2.3 to list: [2.3]
From 2.3 to tuple: (2.3,)
From 2.3 to str: 2.3
From 2.3 to dict: {2.3: 2.3}
From 2.3 to set: {2.3}
From 2.3 to numpy.ndarray: 2.3


In [ ]:
for n in [[0,0.1],(0,0.1),'01',{0:0.1},{0,0.1},np.array([0,0.1])]:
    for t in types:
        print(f'From {n} to {typestr(t)}: {cast(n,t)}')

From [0, 0.1] to int: [0, 0]
From [0, 0.1] to float: [0.0, 0.1]
From [0, 0.1] to list: [0, 0.1]
From [0, 0.1] to tuple: (0, 0.1)
From [0, 0.1] to str: [0, 0.1]
From [0, 0.1] to dict: [0, 0.1]
From [0, 0.1] to set: {0, 0.1}
From [0, 0.1] to numpy.ndarray: [0.  0.1]
From (0, 0.1) to int: (0, 0)
From (0, 0.1) to float: (0.0, 0.1)
From (0, 0.1) to list: [0, 0.1]
From (0, 0.1) to tuple: (0, 0.1)
From (0, 0.1) to str: (0, 0.1)
From (0, 0.1) to dict: (0, 0.1)
From (0, 0.1) to set: {0, 0.1}
From (0, 0.1) to numpy.ndarray: [0.  0.1]
From 01 to int: [0, 1]
From 01 to float: [0.0, 1.0]
From 01 to list: ['0', '1']
From 01 to tuple: ('0', '1')
From 01 to str: 01
From 01 to dict: 01
From 01 to set: {'1', '0'}
From 01 to numpy.ndarray: 01
From {0: 0.1} to int: {0: 0}
From {0: 0.1} to float: {0: 0.1}
From {0: 0.1} to list: [0]
From {0: 0.1} to tuple: (0,)
From {0: 0.1} to str: {0: 0.1}
From {0: 0.1} to dict: {0: 0.1}
From {0: 0.1} to set: {0}
From {0: 0.1} to numpy.ndarray: {0: 0.1}
From {0, 0.1} to i

This extends the desired behavior of typecasting for iterables and noniterables.

# Conversions

Here we present conversions between common datatypes (like numbers to binary arrays).

In [ ]:
#export
def pad(data : Union[np.ndarray,list],
          bits : Optional[int] = None,
          to : Union[int,float] = int) -> np.ndarray:
    '''
    Pads an array with zeros, up to a length of `bits`.
    '''
    if bits is None:
        bits=0
    else:
        bits=bits-len(data)
    x=[0 for i in range(bits)]+list(data)
    x=np.array(x).astype(to)
    return x

In [ ]:
show_doc(pad)
pad(data=[1,0],bits=5,to=float)

<h4 id="pad" class="doc_header"><code>pad</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>pad</code>(**`data`**:`Union`\[`ndarray`, `list`\], **`bits`**:`Optional`\[`int`\]=*`None`*, **`to`**:`Union`\[`int`, `float`\]=*`int`*)

Pads an array with zeros, up to a length of `bits`.

array([0., 0., 0., 1., 0.])

In [ ]:
#export
def fillar(x : list,fillwith=np.NaN,mask=True):
    '''
    Turn uneven nested lists `x` into arrays `y` substituting
    missing entries using `fillwith` and optionally masking.
    '''
    length = max(map(len, x))
    y=np.array([xi+[fillwith]*(length-len(xi)) for xi in x])
    if mask:
        if np.isfinite(fillwith):
            y=np.ma.masked_equal(y,fillwith)
        else:
            y=np.ma.masked_invalid(y)
    return y

In [ ]:
show_doc(fillar)
fillar([[1],[1,1,1]],fillwith=0,mask=False)

<h4 id="fillar" class="doc_header"><code>fillar</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>fillar</code>(**`x`**:`list`, **`fillwith`**=*`nan`*, **`mask`**=*`True`*)

Turn uneven nested lists `x` into arrays `y` substituting
missing entries using `fillwith` and optionally masking.

array([[1, 0, 0],
       [1, 1, 1]])

In [ ]:
fillar([[1],[1,1,1]],fillwith=0,mask=True)

masked_array(
  data=[[1, --, --],
        [1, 1, 1]],
  mask=[[False,  True,  True],
        [False, False, False]],
  fill_value=0)

In [ ]:
#export
def nbits(x : Union[int,float,list,np.ndarray]) -> int:
    '''
    Return the number of bits required to represent the input `x`. 
    '''
    t=type(x)
    if (t is int) or (t is float):
        bits=np.ceil(np.log2(x+1)).astype(int) if x!=0 else 1
    elif (t is list) or (t is np.ndarray):
        bits=len(x)
    return bits

In [ ]:
show_doc(nbits)
for i in range(11):
    print(f"i={i}, nbits({i})={nbits(i)}")

<h4 id="nbits" class="doc_header"><code>nbits</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>nbits</code>(**`x`**:`Union`\[`int`, `float`, `list`, `ndarray`\])

Return the number of bits required to represent the input `x`. 

i=0, nbits(0)=1
i=1, nbits(1)=1
i=2, nbits(2)=2
i=3, nbits(3)=2
i=4, nbits(4)=3
i=5, nbits(5)=3
i=6, nbits(6)=3
i=7, nbits(7)=3
i=8, nbits(8)=4
i=9, nbits(9)=4
i=10, nbits(10)=4


In [ ]:
#export
def rint(x: Union[int,float,list,np.ndarray]) -> Union[int,np.ndarray]:
    '''
    Typecast rounding to np arrays.
    
    '''
    t=type(x)
    if (t is list) or (t is np.ndarray):
        return np.rint(x).astype(int)
    else:
        return round(x)

In [ ]:
show_doc(rint)
rint(np.array([0.5,0.51]))

<h4 id="rint" class="doc_header"><code>rint</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>rint</code>(**`x`**:`Union`\[`int`, `float`, `list`, `ndarray`\])

Typecast rounding to np arrays.

array([0, 1])

In [ ]:
#export
class RNG:
    '''
    Globally stable random number generator. Initialized with fixed `seed`.
    Contains `normal` and `random` methods, each with an `absval` and `asint` argument,
    which convert to positive values and round to integers respectively.
    
    Example use:
        rng=RNG(seed=0)
        rng.normal(x=0,dx=1,shape=(2,2))
        rng.random(a=0,b=5,shape=(2,2),asint=True,absval=True)
    '''
    def __init__(self,seed : Optional[int] = 0):
        self.rng=np.random.default_rng(seed)

    def typecast(self,
                 y : Union[int,float],
                 absval : bool = False,
                 asint : bool = False) -> Union[int,float,np.ndarray]:
        if absval:
            y=abs(y)
        if asint:
            y=rint(y)
        return y

    def normal(self,
               x : Union[float,int] = 0,
               y : Union[float,int] = 0,
               shape : Optional[tuple] = None,
               absval : bool = False,
               asint : bool = False) -> Union[int,float,np.ndarray]:
        '''
        Draw from a Gaussian distribution with mean `x` and standard deviation `y`.
        If `shape` is not None, return a numpy array of draws.
        '''
        return self.typecast(y=self.rng.normal(loc=x,scale=y,size=shape),absval=absval,asint=asint)

    def random(self,
               x : Union[float,int] = 0,
               y : Union[float,int] = 0,
               shape : Optional[tuple] = None,
               absval : bool = False,
               asint : bool = False) -> Union[int,float,np.ndarray]:
        '''
        Draw from a uniform distribution in the interval [`x`,`y`].
        If `shape` is not None, return a numpy array of draws.
        '''
        return self.typecast((y-x)*self.rng.random(size=shape),absval=absval,asint=asint)

In [ ]:
show_doc(RNG)
rng=RNG(seed=0)
print(rng.random(x=0,y=1,shape=(1,2)))
print(rng.normal(x=100,y=1,shape=None))

<h2 id="RNG" class="doc_header"><code>class</code> <code>RNG</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>RNG</code>(**`seed`**:`Optional`\[`int`\]=*`0`*)

Globally stable random number generator. Initialized with fixed `seed`.
Contains `normal` and `random` methods, each with an `absval` and `asint` argument,
which convert to positive values and round to integers respectively.

Example use:
    rng=RNG(seed=0)
    rng.normal(x=0,dx=1,shape=(2,2))
    rng.random(a=0,b=5,shape=(2,2),asint=True,absval=True)

[[0.63696169 0.26978671]]
100.64042265044328


In [ ]:
#export
def num2ar(x: Union[int,float],
            bits : Optional[int] = None,
            to : Union[int,float] = int) -> np.ndarray:
    '''
    Converts decimal number `x` to array `a` zero-padded with `bits`. 
    '''
    bits=bits or nbits(x) #if None, give default
    form='0'+str(bits)+'b'
    binary=format(int(x),form)
    with warnings.catch_warnings(): #ignore numpy deprecation warning
        warnings.simplefilter("ignore")
        a=np.fromstring(binary,'u1')-ord('0')
    return cast(a,to)

In [ ]:
show_doc(num2ar)
for i in range(11):
    print(f"i={i}, num2ar({i})={num2ar(i)}")

<h4 id="num2ar" class="doc_header"><code>num2ar</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>num2ar</code>(**`x`**:`Union`\[`int`, `float`\], **`bits`**:`Optional`\[`int`\]=*`None`*, **`to`**:`Union`\[`int`, `float`\]=*`int`*)

Converts decimal number `x` to array `a` zero-padded with `bits`. 

i=0, num2ar(0)=[0]
i=1, num2ar(1)=[1]
i=2, num2ar(2)=[1 0]
i=3, num2ar(3)=[1 1]
i=4, num2ar(4)=[1 0 0]
i=5, num2ar(5)=[1 0 1]
i=6, num2ar(6)=[1 1 0]
i=7, num2ar(7)=[1 1 1]
i=8, num2ar(8)=[1 0 0 0]
i=9, num2ar(9)=[1 0 0 1]
i=10, num2ar(10)=[1 0 1 0]


In [ ]:
num2ar(10,bits=5,to=float)

array([0., 1., 0., 1., 0.])

In [ ]:
#export
def ar2num(a : Union[list,np.ndarray],
            to : Union[int,float] = int):
    '''
    Converts array `a` to decimal number `x`.
    '''
    temp=str()
    a=np.array(a).astype(int)
    for c in a:
        temp+=str(c)
    x=int(temp,2)
    return cast(x,to)

In [ ]:
show_doc(ar2num)
for i in range(4):
    print(f"a={num2ar(i)}, ar2num(a)={ar2num(num2ar(i))}")

<h4 id="ar2num" class="doc_header"><code>ar2num</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>ar2num</code>(**`a`**:`Union`\[`list`, `ndarray`\], **`to`**:`Union`\[`int`, `float`\]=*`int`*)

Converts array `a` to decimal number `x`.

a=[0], ar2num(a)=0
a=[1], ar2num(a)=1
a=[1 0], ar2num(a)=2
a=[1 1], ar2num(a)=3


In [ ]:
#export
def ar2hex(a : Union[list,np.ndarray],
            bits : Optional[int] = None,
            prefix : bool = True) -> str:
    '''
    Converts binary array to hex string
    in:
        a (numpy array) : binary array to convert
    out:
        h (str) : hex conversion of a 
    '''
    bits=bits or nbits(a)
    form='0'+str(int(np.log2(bits)))+'x'
    h=format(ar2num(a),form)
    if prefix:
        h='0x'+h
    return h

In [ ]:
show_doc(ar2hex)
ar2hex(num2ar(10))

<h4 id="ar2hex" class="doc_header"><code>ar2hex</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>ar2hex</code>(**`a`**:`Union`\[`list`, `ndarray`\], **`bits`**:`Optional`\[`int`\]=*`None`*, **`prefix`**:`bool`=*`True`*)

Converts binary array to hex string
in:
    a (numpy array) : binary array to convert
out:
    h (str) : hex conversion of a 

'0x0a'

In [ ]:
ar2hex(num2ar(10),prefix=False)

'0a'

In [ ]:
#export
def hex2ar(h : str,
            bits : Optional[int] = None,
            to : Union[int,float] = int) -> np.ndarray:
    '''
    Converts a hex string `h` into an array `a` padded with `bits` and elements of `astype`.
    '''
    x=int(h,16)
    a=num2ar(x,bits,to)
    return a

In [ ]:
show_doc(hex2ar)
hex2ar('a')

<h4 id="hex2ar" class="doc_header"><code>hex2ar</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>hex2ar</code>(**`h`**:`str`, **`bits`**:`Optional`\[`int`\]=*`None`*, **`to`**:`Union`\[`int`, `float`\]=*`int`*)

Converts a hex string `h` into an array `a` padded with `bits` and elements of `astype`.

array([1, 0, 1, 0])

In [ ]:
hex2ar('0xa')

array([1, 0, 1, 0])

In [ ]:
#export
def str2ar(s : str,
            to : Union[list,np.ndarray] = np.ndarray) -> Union[list,np.ndarray]:
    '''
    Converts an input string `s` into an array or list `a` as per `astype`.
    '''
    a=cast([int(i) for i in s],to)
    return a

In [ ]:
show_doc(str2ar)
str2ar('111')

<h4 id="str2ar" class="doc_header"><code>str2ar</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>str2ar</code>(**`s`**:`str`, **`to`**:`Union`\[`list`, `ndarray`\]=*`ndarray`*)

Converts an input string `s` into an array or list `a` as per `astype`.

array([1, 1, 1])

In [ ]:
#export
def ar2str(a : Union[list,np.ndarray], to : data = None) -> str:
    '''
    Converts an input array `a` into a string `s`.
    '''
    s=''.join([str(cast(i,to)) for i in a])
    return s

In [ ]:
show_doc(ar2str)
ar2str([1.5,1.5,1.3],int)

<h4 id="ar2str" class="doc_header"><code>ar2str</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>ar2str</code>(**`a`**:`Union`\[`list`, `ndarray`\], **`to`**:`Union`\[`int`, `float`, `list`, `tuple`, `str`, `dict`, `set`, `ndarray`\]=*`None`*)

Converts an input array `a` into a string `s`.

'111'

In [ ]:
#export
def COPY(x : Union[int,float],
         to : Union[int,float] = int) -> Union[int,float]:
    '''
    Simply returns `x`. 
    '''
    return cast(x,to)

def NOT(x : Union[int,float],
        to : Union[int,float] = int) -> Union[int,float]:
    '''
    Return conjugate of `x`. 
    '''
    return cast(1.-x,to)

def AND(x : Union[int,float],
        y : Union[int,float],
        to : Union[int,float] = int) -> Union[int,float]:
    '''
    Return logical AND of `x` and `y`.
    '''
    return cast(x*y,to)

def OR(x : Union[int,float],
       y : Union[int,float],
       to : Union[int,float] = int) -> Union[int,float]:
    '''
    Return logical OR of `x` and `y`. See DeMorgan's Laws.
    '''
    return cast(x+y-x*y,to)

def Exclusive_OR(x : Union[int,float],
                 y : Union[int,float],
                 to : Union[int,float] = int) -> Union[int,float]:
    '''
    Return logical exclusive OR of `x` and `y`. See DeMorgan's Laws.
    '''
    return cast( OR( AND( x , NOT(y,to), to ) , AND ( NOT(x,to) , y, to), to ) , to)

def XOR(*args : Union[int,float,list,np.ndarray],
        to : Union[int,float] = int) -> Union[int,float]:
    '''
    Arbitrary input XOR using recursiveness.
    '''
    x=0.0
    for a in args:
        x=Exclusive_OR(x,a,to)
    return cast(x,to)

In [ ]:
show_doc(NOT)
print(NOT(1))
NOT(0.25,to=float)

<h4 id="NOT" class="doc_header"><code>NOT</code><a href="__main__.py#L9" class="source_link" style="float:right">[source]</a></h4>

> <code>NOT</code>(**`x`**:`Union`\[`int`, `float`\], **`to`**:`Union`\[`int`, `float`\]=*`int`*)

Return conjugate of `x`. 

0


0.75

In [ ]:
show_doc(AND)
print(AND(1,1))
AND(1,0.5,to=float)

<h4 id="AND" class="doc_header"><code>AND</code><a href="__main__.py#L16" class="source_link" style="float:right">[source]</a></h4>

> <code>AND</code>(**`x`**:`Union`\[`int`, `float`\], **`y`**:`Union`\[`int`, `float`\], **`to`**:`Union`\[`int`, `float`\]=*`int`*)

Return logical AND of `x` and `y`.

1


0.5

In [ ]:
for x,y in np.ndindex((2,2)):
    for z in [AND,OR,XOR]:
        print(f"{z.__name__}{x,y}={z(x,y)}")

AND(0, 0)=0
OR(0, 0)=0
XOR(0, 0)=0
AND(0, 1)=0
OR(0, 1)=1
XOR(0, 1)=1
AND(1, 0)=0
OR(1, 0)=1
XOR(1, 0)=1
AND(1, 1)=1
OR(1, 1)=1
XOR(1, 1)=0


In [ ]:
show_doc(XOR)
XOR(1,1,1)

<h4 id="XOR" class="doc_header"><code>XOR</code><a href="__main__.py#L40" class="source_link" style="float:right">[source]</a></h4>

> <code>XOR</code>(**\*`args`**:`Union`\[`int`, `float`, `list`, `ndarray`\], **`to`**:`Union`\[`int`, `float`\]=*`int`*)

Arbitrary input XOR using recursiveness.

1

In [ ]:
XOR(0.8,0.1,0.6,to=float)

0.40709488639999997

In [ ]:
#export
def ar2gr(binary : Union[list,np.ndarray],
             to : Union[list,np.ndarray] = np.ndarray) -> Union[list,np.ndarray]:
    '''
    Converts an input binary array to graycode.
    '''
    binary = cast(binary,int)
    gray = []
    gray += [binary[0]]
    for i in range(1,len(binary)):
        gray += [XOR(binary[i - 1], binary[i])]
    return cast(cast(gray,int),to)

In [ ]:
show_doc(ar2gr)
ar2gr(num2ar(10))

<h4 id="ar2gr" class="doc_header"><code>ar2gr</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>ar2gr</code>(**`binary`**:`Union`\[`list`, `ndarray`\], **`to`**:`Union`\[`list`, `ndarray`\]=*`ndarray`*)

Converts an input binary array to graycode.

array([1, 1, 1, 1])

In [ ]:
#export
def gr2ar(gray : Union[list,np.ndarray],
             to : Union[list,np.ndarray] = np.ndarray) -> Union[list,np.ndarray]:
    '''
    Converts a gray-code array into binary.
    '''
    binary = []
    binary += [gray[0]]
    for i in range(1, len(gray)):
        if (gray[i] == 0):
            binary += [binary[i - 1]]
        else:
            binary += [NOT(binary[i - 1])]
    return cast(cast(binary,int),to)


In [ ]:
show_doc(gr2ar)
gr2ar(ar2gr(num2ar(10)))

<h4 id="gr2ar" class="doc_header"><code>gr2ar</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>gr2ar</code>(**`gray`**:`Union`\[`list`, `ndarray`\], **`to`**:`Union`\[`list`, `ndarray`\]=*`ndarray`*)

Converts a gray-code array into binary.

array([1, 0, 1, 0])

In [ ]:
#export
def num2gr(x:int,to:data=None):
    '''
    Converts decimal number `x` to equivalent gray-code number.
    '''
    return int(ar2str(ar2gr(num2ar(x))),2)

In [ ]:
show_doc(num2gr)
for i in range(11):
    print(f"i={i}, num2gr(i)={num2gr(i)}")

<h4 id="num2gr" class="doc_header"><code>num2gr</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>num2gr</code>(**`x`**:`int`, **`to`**:`Union`\[`int`, `float`, `list`, `tuple`, `str`, `dict`, `set`, `ndarray`\]=*`None`*)

Converts decimal number `x` to equivalent gray-code number.

i=0, num2gr(i)=0
i=1, num2gr(i)=1
i=2, num2gr(i)=3
i=3, num2gr(i)=2
i=4, num2gr(i)=6
i=5, num2gr(i)=7
i=6, num2gr(i)=5
i=7, num2gr(i)=4
i=8, num2gr(i)=12
i=9, num2gr(i)=13
i=10, num2gr(i)=15


In [ ]:
#export            
def convert(obj : Union[int,float,list,hex,str,np.ndarray],
            to : Union[int,float,list,hex,str,np.ndarray] = np.ndarray,
            bits : Optional[int] = None,
            astype : Union[int,float,list,np.ndarray] = int,
            gray : bool = False):
    '''
    Converts an input `obj` into an output of type `to`, padding with `bits`.
    Internally converts `obj` to np.ndarray with elements of dtype `astype`,
    before converting to the desired dtype `to`. If `gray`, first converts this
    binary array to gray-code. If input or output are `hex`, requires prefix of `0x`.
    
    Possible conversions:
        int -> float
        int -> str
        int -> list 
        int -> array
        int -> hex
        
        str -> int
        str -> float
        str -> list
        str -> array
        str -> hex
        
        list -> arr
        list -> int
        list -> float
        list -> str
        list -> hex
        
        arr -> list
        arr -> int
        arr -> float
        arr -> str
        arr -> hex
        
        hex -> int
        hex -> float
        hex -> arr
        hex -> list
        hex -> str
        
    '''
    
    t=type(obj)
    #first convert to binary numpy array
    if (t is np.ndarray) or (t is list):
        x=obj
    elif (t is int) or (t is float):
        x=num2ar(obj,bits,astype)
    else:# t is str
        if obj[:2]=='0x': #obj is hex
            x=hex2ar(obj[2:],bits,astype) 
        else:
            x=str2ar(obj)
    x=cast(pad(x,bits),astype)
    g=cast(ar2gr(x),astype)
    #convert
    if (to is np.ndarray):
        if gray:
            return g
        else:
            return x
    elif (to is list) or (to is set):
        if gray:
            return to(g)
        else:
            return to(x)
    elif (to is int) or (to is float) or (to is hex):
        if gray:
            return to(ar2num(g))
        else:
            return to(ar2num(x))
    else:# to is str
        if gray:
            return ar2str(g)
        else:
            return ar2str(x)

In [ ]:
show_doc(convert)

<h4 id="convert" class="doc_header"><code>convert</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>convert</code>(**`obj`**:`Union`\[`int`, `float`, `list`, `hex`, `str`, `ndarray`\], **`to`**:`Union`\[`int`, `float`, `list`, `hex`, `str`, `ndarray`\]=*`ndarray`*, **`bits`**:`Optional`\[`int`\]=*`None`*, **`astype`**:`Union`\[`int`, `float`, `list`, `ndarray`\]=*`int`*, **`gray`**:`bool`=*`False`*)

Converts an input `obj` into an output of type `to`, padding with `bits`.
Internally converts `obj` to np.ndarray with elements of dtype `astype`,
before converting to the desired dtype `to`. If `gray`, first converts this
binary array to gray-code. If input or output are `hex`, requires prefix of `0x`.

Possible conversions:
    int -> float
    int -> str
    int -> list 
    int -> array
    int -> hex
    
    str -> int
    str -> float
    str -> list
    str -> array
    str -> hex
    
    list -> arr
    list -> int
    list -> float
    list -> str
    list -> hex
    
    arr -> list
    arr -> int
    arr -> float
    arr -> str
    arr -> hex
    
    hex -> int
    hex -> float
    hex -> arr
    hex -> list
    hex -> str
    

In [ ]:
for i in [10,10.0,'10','0xa']:
    t_in=str(type(i)).split('<')[-1].split('>')[0].split('class')[-1].split('\'')[1]
    for j in [int,float,list,str,hex]:#,set,dict]:
        t_out='hex' if j==hex else str(j).split('<')[-1].split('>')[0].split('class')[-1].split('\'')[1]
        print(f'From {t_in} to {t_out}: convert({i},{t_out})={convert(i,j)}')

From int to int: convert(10,int)=10
From int to float: convert(10,float)=10.0
From int to list: convert(10,list)=[1, 0, 1, 0]
From int to str: convert(10,str)=1010
From int to hex: convert(10,hex)=0xa
From float to int: convert(10.0,int)=10
From float to float: convert(10.0,float)=10.0
From float to list: convert(10.0,list)=[1, 0, 1, 0]
From float to str: convert(10.0,str)=1010
From float to hex: convert(10.0,hex)=0xa
From str to int: convert(10,int)=2
From str to float: convert(10,float)=2.0
From str to list: convert(10,list)=[1, 0]
From str to str: convert(10,str)=10
From str to hex: convert(10,hex)=0x2
From str to int: convert(0xa,int)=10
From str to float: convert(0xa,float)=10.0
From str to list: convert(0xa,list)=[1, 0, 1, 0]
From str to str: convert(0xa,str)=1010
From str to hex: convert(0xa,hex)=0xa


In [ ]:
for i in [10,10.0,'10','0xa']:#,{0,10},{0:10,'a':11}]:
    t_in=str(type(i)).split('<')[-1].split('>')[0].split('class')[-1].split('\'')[1]
    for j in [int,float,list,str,hex]:#,set,dict]:
        t_out='hex' if j==hex else str(j).split('<')[-1].split('>')[0].split('class')[-1].split('\'')[1]
        print(f'Gray Code: From {t_in} to {t_out}: convert({i},{t_out})={convert(i,j,gray=True)}')

Gray Code: From int to int: convert(10,int)=15
Gray Code: From int to float: convert(10,float)=15.0
Gray Code: From int to list: convert(10,list)=[1, 1, 1, 1]
Gray Code: From int to str: convert(10,str)=1111
Gray Code: From int to hex: convert(10,hex)=0xf
Gray Code: From float to int: convert(10.0,int)=15
Gray Code: From float to float: convert(10.0,float)=15.0
Gray Code: From float to list: convert(10.0,list)=[1, 1, 1, 1]
Gray Code: From float to str: convert(10.0,str)=1111
Gray Code: From float to hex: convert(10.0,hex)=0xf
Gray Code: From str to int: convert(10,int)=3
Gray Code: From str to float: convert(10,float)=3.0
Gray Code: From str to list: convert(10,list)=[1, 1]
Gray Code: From str to str: convert(10,str)=11
Gray Code: From str to hex: convert(10,hex)=0x3
Gray Code: From str to int: convert(0xa,int)=15
Gray Code: From str to float: convert(0xa,float)=15.0
Gray Code: From str to list: convert(0xa,list)=[1, 1, 1, 1]
Gray Code: From str to str: convert(0xa,str)=1111
Gray Code:

In [ ]:
#hide
notebook2script()

Converted 00_utils.ipynb.
Converted 01_conversion.ipynb.
Converted 02_recursion.ipynb.
Converted 03_templates.ipynb.
Converted index.ipynb.
